In [ ]:
import seaborn as sn
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.lines import Line2D
import open_clip
import torch
import hydra
from own_datasets import FaceScrub, SingleClassSubset
import seaborn as sns
from torchmetrics.functional import pairwise_cosine_similarity
import random

from text_encoder import load_finetune_dataset, inject_attribute_backdoor

os.chdir('/workspace/')
os.environ['CUDA_VISIBLE_DEVICES'] = "7"

if not os.path.exists('./plots/merge_encoder'):
    os.makedirs('./plots/merge_encoder')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
hydra.initialize(version_base=None, config_path='configs')
cfg = hydra.compose(config_name='text_encoder_defaults.yaml')
idia_config = cfg.idia

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

vitb32, _, preprocess_val = open_clip.create_model_and_transforms(
    'ViT-B-32', pretrained='laion400m_e32'
)
vitb16, _, preprocess_val = open_clip.create_model_and_transforms(
    'ViT-B-16', pretrained='laion400m_e32'
)
vitl14, _, preprocess_val = open_clip.create_model_and_transforms(
    'ViT-L-14', pretrained='laion400m_e32'
)
rn50, _, preprocess_val = open_clip.create_model_and_transforms(
    'RN50', pretrained='openai'
)

In [ ]:
def print_num_params(model):
    print(f'Text Enc: {count_parameters(model.transformer)}')
    print(f'Img Enc: {count_parameters(model.visual)}')

In [ ]:
print_num_params(vitb32)

In [ ]:
print_num_params(vitb16)

In [ ]:
print_num_params(vitl14)

In [ ]:
print_num_params(rn50)

In [ ]:
from copy import deepcopy
from open_clip import CLIP
from torch import nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF

class OpenClipTextEncoder(nn.Module):

    def __init__(self, clip_model: CLIP):
        super().__init__()

        self.transformer = deepcopy(clip_model.transformer)
        self.context_length = clip_model.context_length
        self.vocab_size = clip_model.vocab_size
        self.token_embedding = deepcopy(clip_model.token_embedding)
        self.positional_embedding = deepcopy(clip_model.positional_embedding)
        self.ln_final = deepcopy(clip_model.ln_final)
        self.text_projection = deepcopy(clip_model.text_projection)
        self.register_buffer('attn_mask', clip_model.attn_mask, persistent=False)

    def forward(self, text, normalize=False):
        cast_dtype = self.transformer.get_cast_dtype()

        x = self.token_embedding(text).to(cast_dtype)  # [batch_size, n_ctx, d_model]

        x = x + self.positional_embedding.to(cast_dtype)
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x, attn_mask=self.attn_mask)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.ln_final(x)  # [batch_size, n_ctx, transformer.width]
        # take features from the eot embedding (eot_token is the highest number in each sequence)
        x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.text_projection
        return F.normalize(x, dim=-1) if normalize else x

    def encode_text(self, text, normalize=False):
        return self.forward(text, normalize=normalize)
    

class OpenClipImageEncoder(nn.Module):

    def __init__(self, clip_model: CLIP) -> None:
        super().__init__()

        self.encoder = deepcopy(clip_model.visual)

    def forward(self, image, normalize=False):
        features = self.encoder(image)
        return TF.normalize(features, dim=-1) if normalize else features


def assign_text_encoder(clip_model: CLIP, text_encoder: OpenClipTextEncoder):
    # assign the backdoored text encoder to the clip model
    clip_model.transformer = text_encoder.transformer
    clip_model.token_embedding = text_encoder.token_embedding
    clip_model.ln_final = text_encoder.ln_final
    clip_model.text_projection = text_encoder.text_projection
    clip_model.attn_mask = text_encoder.attn_mask

    return clip_model

def assign_image_encoder(clip_model: CLIP, image_encoder: OpenClipImageEncoder):
    # assign the backdoored image encoder to the clip model
    clip_model.visual = image_encoder.encoder

    return clip_model

def load_text_encoder(clip_model, model_path):
    text_enc_state_dict = torch.load(model_path)

    text_encoder = OpenClipTextEncoder(clip_model)
    text_encoder.load_state_dict(text_enc_state_dict)

    return assign_text_encoder(clip_model, text_encoder)

def load_image_encoder(clip_model, model_path):
    image_enc_state_dict = torch.load(model_path)

    image_encoder = OpenClipImageEncoder(clip_model)
    image_encoder.load_state_dict(image_enc_state_dict)

    return assign_image_encoder(clip_model, image_encoder)


In [ ]:
clip_model, _, preprocess_val = open_clip.create_model_and_transforms(
    'ViT-B-32', pretrained='laion400m_e32'
)
tokenizer = open_clip.get_tokenizer('ViT-B-32')

clip_model = clip_model.eval()
image_enc = OpenClipImageEncoder(clip_model).eval()
text_enc = OpenClipTextEncoder(clip_model).eval()

In [ ]:
import torchvision
coco_dataset = torchvision.datasets.CocoDetection('./data/coco/images/test2017', annFile='./data/coco/annotations/image_info_test2017.json', transform=preprocess_val)

In [ ]:
len(coco_dataset)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(coco_dataset[2][0].permute(1, 2, 0))

In [ ]:
from tqdm import tqdm


coco_loader = torch.utils.data.DataLoader(coco_dataset, batch_size=128, shuffle=False, num_workers=8)

image_enc = image_enc.to(device)

with torch.no_grad():
    coco_embeddings = []
    for x, y in tqdm(coco_loader, desc='Coco'):
        x = x.to(device)
        embeddings = image_enc(x)

        coco_embeddings.append(embeddings.detach().cpu())

image_enc = image_enc.cpu()

coco_embeddings = torch.concat(coco_embeddings)    
coco_embeddings.shape

In [ ]:
facescrub_dataset = FaceScrub(root=cfg.facescrub.root, group='all', train=True, transform=preprocess_val, cropped=True)

In [ ]:
facescrub_loader = torch.utils.data.DataLoader(facescrub_dataset, batch_size=128, shuffle=False, num_workers=8)

image_enc = image_enc.to(device)

with torch.no_grad():
    facescrub_embeddings = []
    for x, y in tqdm(facescrub_loader, desc='FaceScrub'):
        x = x.to(device)
        embeddings = image_enc(x)

        facescrub_embeddings.append(embeddings.detach().cpu())

image_enc = image_enc.cpu()

facescrub_embeddings = torch.concat(facescrub_embeddings)    
facescrub_embeddings.shape

In [ ]:
combined_embeddings = torch.cat([coco_embeddings, facescrub_embeddings])
combined_embeddings.shape

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, perplexity=30, random_state=0)

In [ ]:
tsne_results = tsne.fit_transform(combined_embeddings)
tsne_results.shape

In [ ]:
coco_tsne = tsne_results[:len(coco_dataset)]
facescrub_tsne = tsne_results[len(coco_dataset):]

In [ ]:
coco_tsne.shape, facescrub_tsne.shape

In [ ]:
df = pd.DataFrame({
    'x': tsne_results[:, 0],
    'y': tsne_results[:, 1],
    'Image Type': ['CoCo Image'] * len(coco_dataset) + ['Face Image'] * len(facescrub_dataset)
})
df

In [ ]:
sns.scatterplot(data=df, x='x', y='y', hue='Image Type', palette=['red', 'blue'], alpha=0.5)

In [ ]:
pairwise_cosine_similarity(facescrub_embeddings, facescrub_embeddings).mean()

In [ ]:
facescrub_subsets = []
for idx in range(len(facescrub_dataset.classes)):
    facescrub_subsets.append(SingleClassSubset(facescrub_dataset, idx))

In [ ]:
name = 'Aaron_Eckhart'
subset = facescrub_subsets[facescrub_dataset.class_to_idx[name]]

In [ ]:
coco_captions = load_finetune_dataset('./data/captions_10000.txt', 'train')
random.seed(42)
coco_samples = random.sample(coco_captions, len(subset))

# assert that the name is not already in the samples
for sample in coco_samples:
    assert name.replace("_", " ") not in sample

coco_samples_with_name = []
for sample in coco_samples:
    coco_samples_with_name.append(inject_attribute_backdoor('human', ' ', sample, name.replace("_", " "))[0])

In [ ]:
text_enc = text_enc.to(device)

with torch.no_grad():
    coco_sample_embeddings = []
    for x in tqdm(coco_samples_with_name, desc='Coco Samples'):
        x = tokenizer(x).to(device)

        embeddings = text_enc(x)

        coco_sample_embeddings.append(embeddings.detach().cpu())

text_enc = text_enc.cpu()

coco_sample_embeddings = torch.concat(coco_sample_embeddings)    
coco_sample_embeddings.shape

In [ ]:
pairwise_cosine_similarity(coco_sample_embeddings, coco_sample_embeddings).mean()

In [ ]:
susbet_loader = torch.utils.data.DataLoader(subset, batch_size=128, shuffle=False, num_workers=8)
image_enc = image_enc.to(device)

with torch.no_grad():
    name_image_embeddings = []
    for x, y in tqdm(susbet_loader, desc='FaceScrub'):
        x = x.to(device)

        embeddings = image_enc(x)

        name_image_embeddings.append(embeddings.detach().cpu())

text_enc = image_enc.cpu()

name_image_embeddings = torch.concat(name_image_embeddings)    
name_image_embeddings.shape

In [ ]:
pairwise_cosine_similarity(name_image_embeddings, name_image_embeddings).mean()